In [ ]:
import numpy as np
import zebende as zb
import pandas as pd
from pathlib import Path

In [ ]:
# criando um vetor com as jalenas temporais
tws = np.array([4, 9, 28, 60, 104, 407, 4564, 35212, 86015, 252313], dtype=np.int64)[:2]
tws.size

In [ ]:
# criando uma fila de arquivos a serem processados
root = Path()
# data folder
data_dir = root / 'data_eeg'
# 'file path'/name.txt generator
file_gen = data_dir.glob('*.txt')

In [4]:
for i in file_gen:
    print( 'processing file: {} ...'.format(i.name))
    # carregando o arquivo selecionado
    data = pd.read_csv(i)
    # filtrando as series que serão processadas 
    data = data.iloc[:, 1:30]
    # convertendo em uma matriz numpy
    data_np = data.to_numpy()
    print('\tdata shape: {}\n\tTime window scale size: {}'.format(data_np.shape, tws.size))
    # criando a série integrada
    int_data = zb.integrated_series(data_np)
    # calculando o dfa
    dfa = zb.dfa(int_data, tws)
    # Colocando as escalas temporais ao lado dos cálculos do dfa. 
    dfa_concat = np.c_[tws, dfa]
    # gravando os resultados no sistema de arquivos
    pd_dfa = pd.DataFrame(dfa_concat)
    # renomeando coluna 0 para tws
    pd_dfa.rename(columns={0: "tws"}, inplace=True)
    # mudando tipo de dado para a coluna tws
    pd_dfa = pd_dfa.astype({"tws": 'int64'})
    # Criando path e nome do arquivo do resultado
    out_file_path = data_dir / 'dfa_out_{}.dat'.format(i.stem)
    # gravando dataframe
    pd_dfa.to_csv(out_file_path, sep=' ', index=False)
    print('file {} processed!'.format(i.name))